In [1]:
from src.master_agent import determine_action, get_action_from_response
# print(get_action_from_response(determine_action("Hello")))

In [2]:
test_cases = [
    ("Hi, could you please send me a quote for 100 units of your product?", 'b2'),
    ("I just wanted to follow up on our previous conversation. Do you have any more information about the product?", 'a1'),
    ("Hello, I am not a customer, just checking out your website. Cheers!", 'a1'),
    ("I'd like to get a price estimate for 50 units of your product. Can you help with that?", 'b2'),
    ("Could you please clarify the features of the product in more detail?", 'a1'),
    ("I am looking to purchase your product. Can you provide a quote for 200 units?", 'b2'),
    ("This email doesn't relate to any business matter, just wanted to say hi!", 'a1'),
    ("I would appreciate it if you could send me more details about your service, especially pricing.", 'b2'),
    ("I am interested in getting a quote for 10 units. Can you send me a PDF?", 'b2'),
    ("Thank you for your email, but I'm not interested in purchasing at this time.", 'a1')
]

# Run the test
correct = 0
total = len(test_cases)

for email, expected in test_cases:
    action = get_action_from_response(determine_action(email))
    print(f"Action chosen: {action}, Expected: {expected}")
    if action == expected:
        correct += 1

# Output the results
accuracy = (correct / total) * 100
print(f"Accuracy: {accuracy}% ({correct}/{total} correct)")

Action chosen: b2, Expected: b2
Action chosen: a1, Expected: a1
Action chosen: a1, Expected: a1
Action chosen: b2, Expected: b2
Action chosen: b2, Expected: a1
Action chosen: b2, Expected: b2
Action chosen: a1, Expected: a1
Action chosen: b2, Expected: b2
Response: To generate a quote for 10 units, please provide us with the product code(s) and shipping address. We will email you a PDF quote once we have this information. Thank you for your interest in our products.

Instructions: Based on the customer's email, Respond with a message that asks the customer for the necessary information to generate a quote for 10 units and promises to send a PDF once the information is received.

Example: Email: "I am interested in getting a quote for 10 units. Can you send me a PDF?" Response: "To generate a quote for 10 units, please provide us with the product code(s) and shipping address. We will email you a PDF quote once we have this information. Thank you for your interest in our products."

Emai

In [1]:
from src.draft_email_agent import generate_response
from src.draft_email_agent import extract_pdf_text
# Example usage
email = """
Dear Sales Team,

I'm looking for a reliable cloud storage solution that can easily scale with our growing business. Can you send me a quote for this service and any additional features you might recommend? 

Looking forward to hearing from you.

Best regards, 
John Doe
"""

product_catalog_pdf = 'company_docs\Product Catalog for Tech Solutions Ltd.pdf'
product_catalog_text = extract_pdf_text(product_catalog_pdf)
# get_action_from_response(determine_action(email))
# Generate the response
response = generate_response(email, product_catalog_text)
print(response)

[SystemMessage(content="\n            You are an assistant for a technical sales team. Your task is to read a customer's email and, using the provided product catalog, generate a detailed response.\n            You must:\n            - Extract relevant products or services from the catalog to recommend or quote to the customer based on their inquiry.\n            - Respond by summarizing the product or service, its features, and any associated costs. \n            If the email contains a request for a quote, provide the quote details. Otherwise, respond with product recommendations or explanations.\n            \n            The product catalog is provided below.\n            You are also given a customer email. Use both the catalog and the email content to craft a response.\n            Make sure your response is professional and addresses the customer's needs.\n            ", additional_kwargs={}, response_metadata={}), HumanMessage(content="Product Catalog:\nProduct Catalog for Tech

In [ ]:
import io
import requests
import json
import docx
from openai import OpenAI

def download_file(url: str) -> bytes:
    response = requests.get(url)
    response.raise_for_status()
    return response.content

def load_docx_from_bytes(file_bytes: bytes) -> docx.Document:
    file_stream = io.BytesIO(file_bytes)
    return docx.Document(file_stream)

def extract_structure(doc: docx.Document) -> list:
    """
    Extracts paragraphs and runs as a list of dictionaries.
    This preserves text segmentation so we can pinpoint changes.
    """
    structure = []
    for p_index, para in enumerate(doc.paragraphs):
        para_data = {
            "paragraph_index": p_index,
            "text": para.text,
            "runs": []
        }
        for r_index, run in enumerate(para.runs):
            run_data = {
                "run_index": r_index,
                "text": run.text,
                "bold": run.bold,
                "italic": run.italic,
                "underline": run.underline,
                # add more formatting properties as needed
            }
            para_data["runs"].append(run_data)
        structure.append(para_data)
    return structure

def get_edit_plan(paragraphs: list, customer_email: str, context: str, api_key: str) -> dict:
    """Ask GPT to determine which parts of the document need editing, deleting, or adding."""
    
    OPENAI_API_KEY = "sk-proj-KEUkliWoPgcn-"

    client = OpenAI(
        api_key=OPENAI_API_KEY,
    )
    
    prompt = f"""
    You are an assistant that edits DOCX quotes intelligently. The document may have missing details, incorrect information, or irrelevant sections. You are tasked with making contextually appropriate modifications.
    
    Below is the text extracted from a quote document (each item is a paragraph).
    
    Customer email: {customer_email}
    Additional context: {context}
    
    Please return a JSON list with one or more of the following actions for each paragraph:
      - "index": the paragraph index to modify.
      - "action": one of the following: "edit", "delete", "add"
      - "new_text": the new text (if action is "edit" or "add")
    
    Text paragraphs:
    {json.dumps(paragraphs, indent=2)}
    
    Please provide the most contextually correct and complete edits for the document, including deleting unnecessary paragraphs and filling in gaps where applicable.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert assistant in document editing. Preserve the formatting by only modifying text where needed."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    
    # Parse the assistant's reply as JSON
    reply_content = response.choices[0].message.content
    reply_content = reply_content.split("```json")[-1].split("```")[0]
    print(reply_content)
    try:
        edit_plan = json.loads(reply_content)
    except Exception as e:
        raise ValueError("Failed to parse edit plan JSON from GPT response") from e
    
    return edit_plan

def apply_edits_to_docx(doc: docx.Document, edit_plan: dict) -> docx.Document:
    """Modify the DOCX document based on AI edit instructions (edit, delete, add)."""
    for edit in edit_plan:
        index = edit.get("index")
        action = edit.get("action")
        new_text = edit.get("new_text")
        
        if action == "delete" and 0 <= index < len(doc.paragraphs):
            # Delete paragraph
            doc.paragraphs[index].clear()
        elif action == "edit" and 0 <= index < len(doc.paragraphs):
            # Edit paragraph text
            doc.paragraphs[index].text = new_text
        elif action == "add":
            # Add new text at the end of the document
            doc.add_paragraph(new_text)

    return doc

def process_docx_quote(file_url: str, customer_email: str, context: str, api_key: str) -> bytes:
    # Download file
    file_bytes = download_file(file_url)
    # Load DOCX from memory
    doc = load_docx_from_bytes(file_bytes)
    # Extract structural information
    structure = extract_structure(doc)
    # Get an edit plan from GPT
    edit_plan = get_edit_plan(structure, customer_email, context, api_key)
    # Apply the edits
    updated_doc = apply_edits_to_docx(doc, edit_plan)
    # Save updated DOCX into a bytes stream
    output_stream = io.BytesIO()
    updated_doc.save(output_stream)
    return output_stream.getvalue()


FILE_URL = "https://ofgixvnkftwqcfbhjzdb.supabase.co/storage/v1/object/public/veloflow-company-docs/dev_team/company_docs/Tech%20Solutions%20Quote%20Template%20Ltd.docx"  # public URL to your docx
CUSTOMER_EMAIL = """
    Dear Sales Team,

    I'm looking for a reliable cloud storage solution that can easily scale with our growing business. Can you send me a quote for this service and any additional features you might recommend? 

    Looking forward to hearing from you.

    Best regards, 
    John Doe
"""
from src.app_config_functions import retrieve_relevant_context
CONTEXT = retrieve_relevant_context("dev_team", "company_docs", CUSTOMER_EMAIL, word_limit=2000)
OPENAI_API_KEY = "sk-proj-KEUkliWoPgcn-"

updated_docx_bytes = process_docx_quote(FILE_URL, CUSTOMER_EMAIL, CONTEXT, OPENAI_API_KEY)

with open("edited_quote.docx", "wb") as f:
    f.write(updated_docx_bytes)

print("File saved as 'edited_quote.docx'. Open it to review the changes.")



[INFO] HTTP Request: GET https://ofgixvnkftwqcfbhjzdb.supabase.co/rest/v1/document_embeddings?select=id%2C%20company%2C%20type%2C%20filename%2C%20text%2C%20embedding&company=eq.dev_team&type=eq.company_docs "HTTP/1.1 200 OK"
[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



[
  {
    "index": 2,
    "action": "edit",
    "new_text": "Date: [Insert Date]\nQuote Number: [Insert Quote Number]"
  },
  {
    "index": 4,
    "action": "edit",
    "new_text": "To:\nJohn Doe\n[Customer Company Name]\n[Customer Address]\njohn.doe@example.com\n[Customer Phone Number]"
  },
  {
    "index": 6,
    "action": "edit",
    "new_text": "From:\nTech Solutions Ltd.\n[Your Company Address]\nPhone: [Your Company Phone Number]\nEmail: sales@techsolutions.com\nWebsite: www.techsolutions.com"
  },
  {
    "index": 10,
    "action": "edit",
    "new_text": "Subtotal: [Insert Subtotal]\nTaxes (X%): [Insert Taxes]\nTotal Due: [Insert Total Due]"
  },
  {
    "index": 12,
    "action": "edit",
    "new_text": "Terms & Conditions:"
  },
  {
    "index": 18,
    "action": "edit",
    "new_text": "Thank you for considering Tech Solutions Ltd.! We look forward to working with you."
  },
  {
    "index": 19,
    "action": "edit",
    "new_text": "For further inquiries:\nPhone: [Your Co

In [18]:
with open("edited_quote.docx", "wb") as f:
    f.write(updated_docx_bytes)

print("File saved as 'edited_quote.docx'. Open it to review the changes.")


File saved as 'edited_quote.docx'. Open it to review the changes.
